In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
# Drop unused columns
train = train.drop(columns=['Unnamed: 0', 'id'])
test = test.drop(columns=['Unnamed: 0', 'id'])

# Look at the first few rows
train.head()

In [ ]:
train.info()

Here is something very important. We can see that there are 5 object columns, which means we actually need to prepare the data in order to be able to analyze it. Also the 21. Col is a float64 which means increased precision.

### Info Analysis
All columns are complete except the 21. Arrival Delay in Minutes, which is missing 310 entries. Considering the total amount of entries being 103.904 it only accounts for 0.00298 percent. We will still remove it.

In [ ]:
train.dropna(axis=0, inplace=True)

In [ ]:
# Source - https://stackoverflow.com/a
# Posted by Walid
# Retrieved 2026-01-21, License - CC BY-SA 4.0
print(train['Gender'].value_counts())
print()
print(train['Customer Type'].value_counts())
print()
print(train['Type of Travel'].value_counts())
print()
print(train['Class'].value_counts())
print()
print(train['satisfaction'].value_counts())

In [ ]:
mappings = {
    'Gender': {'Male': 1, 'Female': 0},
    'Customer Type': {'Loyal Customer': 1, 'disloyal Customer': 0},
    'Type of Travel': {'Business travel': 1, 'Personal Travel': 0},
    'Class': {'Business': 2, 'Eco Plus': 1, 'Eco': 0},
    'satisfaction': {'satisfied': 1, 'neutral or dissatisfied': 0}
}

for col, mapping in mappings.items():
    train[col] = train[col].map(mapping).fillna(-1).astype(int)

In [ ]:
plt.figure(figsize=(12, 8))
# We only correlate numerical columns
correlation_matrix = train.select_dtypes(include=['number']).corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Feature Correlation Matrix')
plt.show()

### Correlations
The strongest correlations are of course **Departure Delay in Minutes** and **Arrival Delay in Minutes**, due to the fact of them having a clear causality. The more you delay your start, the later you arrive at your destination - which is quite annoying for most people.

The most important thing to keep in mind is that the dataset comes from a survey conducted by the airline
Gate Location correlates with the following: **Inflight wifi service**, **Departure/Arrival time convenient**, **Ease of Online booking**
None the less we couldn't find any casaul connection to these cols.